<a href="https://colab.research.google.com/github/buganart/mocogan/blob/master/mocogan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

# Description


In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
video_db_dir = "/content/drive/My Drive/VIDEOS/SINGLE_FILE" #@param {type:"string"}
# experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}

In [ ]:
#@title Clone mocogan repo

!git clone https://github.com/buganart/mocogan

In [ ]:
#@title Install Dependencies

%cd /content/mocogan
# %pip install -r requirements.txt
%pip install sk-video
clear_on_success("Dependencies installed.")

In [ ]:
#@title Copy video files to runtime

local_wav_dir = Path("/content/mocogan/raw_data")
!find "{video_db_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$local_wav_dir"/
clear_on_success("All files copied to this runtime.")


In [ ]:
#@title RESIZE

import glob
import os
current_path = "/content/mocogan/"
resized_path = os.path.join(current_path, 'resized_data')
files = glob.glob(os.path.join(current_path, 'raw_data/*.*'))

for i, file in enumerate(files):
    resized_filename = os.path.join(resized_path, str(i))
    !ffmpeg -i "{file}" -pix_fmt yuv420p -vf scale=96:96 "{resized_filename}".mp4

# crop=96:96:42:24
clear_on_success("All files resized.")

In [ ]:
#@title TRAIN

# This done a bit weirdly because setting PYTHONPATH=$PWD removes variables afterwards. A colab bug, maybe.
# !env PYTHONPATH="$(pwd)" python scripts/train.py \
# --save_path "$experiment_dir"

!python train.py